In [1]:
# import dependencies/libaries

import pandas as pd
import requests

import xmltodict
import json

### Statewide results

In [2]:
# get the file using the requests library
headers = {
    'User-Agent': 'VTDigger',
    'From': 'erin@vtdigger.org'  # This is another valid field
}

file = requests.get('https://electionresults.vermont.gov/rss/4353/ElectionSummary.xml', headers=headers)
data = file.text

# save it so that I can change the one character
with open('ElectionSummary.xml', 'w+') as fd:
    fd.write(data)

In [2]:
# read it and parse it with xmltodict

with open('ElectionSummary.xml') as fd:
    doc = xmltodict.parse(fd.read())


In [3]:
# from now on, the data is in the form of an incredibly nested dictionary. 

summary = doc['rss']['channel']['ElectionSummary']

#### Dem race

In [5]:
# to access each race, you have to pull out the part of the nested dict where it's hidden.
# for example, here's the gov race:

dem = pd.DataFrame(doc['rss']['channel']['ElectionSummary']['Party'][0]['Office'])

demgov = pd.DataFrame(doc['rss']['channel']['ElectionSummary']['Party'][0]['Office'][1]['Candidate'])

demgov.sort_values(by='Votes', ascending=False)

,CandidateId,Name,Votes,CandidateType,Winner
7,80572,KEVIN HOYT,8,Write-in,False
2,74683,PATRICK WINBURN,7662,Registered,False
4,77891,PHIL SCOTT,6056,Write-in,False
6,89366,ERYNN HAZLETT WHITNEY,6,Write-in,False
32,85038,ROGER KILBOURN,6,Write-in,False
...,...,...,...,...,...
49,76146,SCOTT DOUGLAS,1,Write-in,False
48,84154,MOLLY GRAY,1,Write-in,False
46,77698,ZILLIAN COLASUIDO,1,Write-in,False
45,79766,JIM COLLINS,1,Write-in,False


In [6]:
# liutenant gov:

demlg = pd.DataFrame(doc['rss']['channel']['ElectionSummary']['Party'][0]['Office'][2]['Candidate'])

demlg.sort_values(by='Votes', ascending=False)

,CandidateId,Name,Votes,CandidateType,Winner
3,74926,BRENDA SIEGEL,9945,Registered,False
2,74718,DEBBIE INGRAM,9466,Registered,False
56,78268,BLANK,9,Write-in,False
6,91959,DAVID ZUCKERMAN,59,Write-in,False
8,90470,MEG HANSEN,54,Write-in,False
...,...,...,...,...,...
38,86348,ALEC ELLSWORTH,1,Write-in,False
39,90471,RALPH CORBO,1,Write-in,False
40,87116,ANDREW CHAMPAGNE,1,Write-in,False
41,92666,BEN KINGSLY,1,Write-in,False


In [7]:
# this is what the underlying nested dict looks like. Even after converting into a dataframe, there's a dict within a dict. Fun!
dem.head(10)

,OfficeName,OfficeDistrict,VoteFor,Candidate,WriteInVotes,BlankVotes,Overvotes
0,REPRESENTATIVE TO CONGRESS,None,1,"[{'CandidateId': '74839', 'Name': 'RALPH ""CARC...",237,2844,119
1,GOVERNOR,None,1,"[{'CandidateId': '74840', 'Name': 'RALPH ""CARC...",6533,7995,138
2,LIEUTENANT GOVERNOR,None,1,"[{'CandidateId': '74861', 'Name': 'TIM ASHE', ...",569,5720,76
3,STATE TREASURER,None,1,"[{'CandidateId': '74719', 'Name': 'BETH PEARCE...",392,12793,92
4,SECRETARY OF STATE,None,1,"[{'CandidateId': '74630', 'Name': 'JIM CONDOS'...",358,12063,59
5,AUDITOR OF ACCOUNTS,None,1,"[{'CandidateId': '74632', 'Name': 'DOUG HOFFER...",139,15350,372
6,ATTORNEY GENERAL,None,1,"[{'CandidateId': '74746', 'Name': 'T.J. DONOVA...",823,13702,101
7,STATE SENATOR,ADDISON SENATE,2,"[{'CandidateId': '74941', 'Name': 'CHRISTOPHER...",71,2918,0
8,STATE SENATOR,BENNINGTON SENATE,2,"[{'CandidateId': '74765', 'Name': 'BRIAN CAMPI...",63,2246,2
9,STATE SENATOR,CALEDONIA SENATE,2,"[{'CandidateId': '74984', 'Name': 'MATTHEW CHO...",88,2563,1


In [8]:
dem.columns

Index(['OfficeName', 'OfficeDistrict', 'VoteFor', 'Candidate', 'WriteInVotes',
       'BlankVotes', 'Overvotes'],
      dtype='object')

In [13]:
# demtest = demsendict[demsendict.OfficeDistrict == 'ADDISON SENATE']
# test_con = []
# for n in demtest.Candidate:
#     test_con.append(pd.DataFrame(n))
# pd.concat(test_con)

#### Republican race

In [9]:
# same stuff but for GOP race

gop = pd.DataFrame(doc['rss']['channel']['ElectionSummary']['Party'][2]['Office'])

gopgov = pd.DataFrame(doc['rss']['channel']['ElectionSummary']['Party'][2]['Office'][1]['Candidate'])

gopgov.sort_values(by='Votes', ascending=False).head()

,CandidateId,Name,Votes,CandidateType,Winner
3,74977,EMILY PEYTON,970,Registered,False
0,74967,DOUGLAS CAVETT,966,Registered,False
14,88654,REBECCA HOLCOMBE,9,Write-in,False
2,74720,BERNARD PETERS,772,Registered,False
19,77401,JIM DOUGLAS,7,Write-in,False


In [10]:
# progressives

proggov = pd.DataFrame(doc['rss']['channel']['ElectionSummary']['Party'][1]['Office'][1]['Candidate'])
proggov['Votes'] = pd.to_numeric(proggov['Votes'], errors='coerce')
proggov['Name'] = proggov.Name.str.strip()
proggov.pivot_table(index='Name', values='Votes', aggfunc=sum).sort_values(by='Votes', ascending=False)

,Votes
Name,
DAVID ZUCKERMAN,273
CRIS ERICSON,254
BOOTS WARDINSKI,239
PHIL SCOTT,41
REBECCA HOLCOMBE,5
KEVIN HOYT,3
CHRIS BRIMMER,3
EMILY PEYTON,2
JOHN KLAR,2


In [11]:
goplg = pd.DataFrame(doc['rss']['channel']['ElectionSummary']['Party'][2]['Office'][2]['Candidate'])

goplg.sort_values(by='Votes', ascending=False)

,CandidateId,Name,Votes,CandidateType,Winner
16,76050,KEVIN HOYT,8,Write-in,False
5,84913,MOLLY GRAY,361,Write-in,False
4,74996,DWAYNE TUCKER,3066,Registered,False
46,92882,MOLLY GRAY,3,Write-in,False
43,89781,DONALD TURNER,3,Write-in,False
...,...,...,...,...,...
36,86614,RICHARD SMITH,1,Write-in,False
38,80072,KYLE FISHER,1,Write-in,False
41,77624,ELLEN BRESLEN,1,Write-in,False
42,79523,BUD TUGLEY,1,Write-in,False


In [6]:
# ok so this big mess is how I retrieve the legislative election results. 
# as I said above, everything is nested, so in order to get around that I have to loop through the dict
# and access the inner levels of the nesting within. 
# the rest is just naming the columns of the data, and then declaring/suggesting a winner,
# and then saving it as a csv.


def getrace(party, office):
    all_con = []
    # filter by party
    for n in summary['Party']:
        if n['PartyName'] == party:
            #filter by office
            for district in n['Office']:
                if district['OfficeName'] == office:
                    this_candidate = district['Candidate']
                    if type(this_candidate) is list:
                        for x in this_candidate:
                            this_con = {}
                            this_con['OfficeDistrict'] = district['OfficeDistrict']                                
                            this_con['CandidateId'] = x['CandidateId']
                            this_con['Name'] = x['Name']
                            this_con['Votes'] = x['Votes']
                            this_con['CandidateType'] = x['CandidateType']
                            this_con['Winner'] = x['Winner']
                            all_con.append(pd.DataFrame(this_con, index=['CandidateId']))
                    else:
                        this_con = {}
                        this_con['OfficeDistrict'] = district['OfficeDistrict']                                
                        this_con['CandidateId'] = this_candidate['CandidateId']
                        this_con['Name'] = this_candidate['Name']
                        this_con['Votes'] = this_candidate['Votes']
                        this_con['CandidateType'] = this_candidate['CandidateType']
                        this_con['Winner'] = this_candidate['Winner']
                        all_con.append(pd.DataFrame(this_con, index=['CandidateId']))
    this_df = pd.concat(all_con)
    this_df = this_df[this_df.CandidateId.notnull()].set_index('CandidateId')
    this_df['Votes'] = pd.to_numeric(this_df['Votes'], errors='raise')
    idx = this_df.groupby(['OfficeDistrict'])['Votes'].transform(max) == this_df['Votes']
    winners = this_df[idx]
    winners['Leading candidate'] = 'Leading candidate'
    this_df = this_df.merge(winners, how='left')
#     this_df = this_df[(this_df['Leading candidate'] == 'Leading candidate') | 
#                      (this_df.CandidateType != 'Write-in')]
    this_df = this_df.sort_values(by='Votes', ascending=False)
    this_df.Name = this_df.Name.str.replace('"', '')
    this_df.to_csv("Allcand-" + party + "_" + office + ".csv", index=False)
    return this_df

In [7]:
# getting statewide leg results by party
# don't worry these errors are not as big a deal is they seem

dem_sen = getrace('DEMOCRATIC', 'STATE SENATOR')
gop_sen = getrace('REPUBLICAN', 'STATE SENATOR')
dem_house = getrace('DEMOCRATIC', 'STATE REPRESENTATIVE')
gop_house = getrace('REPUBLICAN', 'STATE REPRESENTATIVE')


<ipython-input-6-96c5a68314da>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winners['Leading candidate'] = 'Leading candidate'
<ipython-input-6-96c5a68314da>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winners['Leading candidate'] = 'Leading candidate'
<ipython-input-6-96c5a68314da>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [79]:
# dem_sen.to_csv('test.csv')

In [99]:
# an example of the results
dem_sen.head()

,OfficeDistrict,Name,Votes,CandidateType,Winner,Leading candidate
1,ADDISON SENATE,RUTH HARDY,6226,Registered,False,Leading candidate
0,ADDISON SENATE,CHRISTOPHER BRAY,5853,Registered,False,NaN
11,BENNINGTON SENATE,BRIAN CAMPION,4923,Registered,False,NaN
12,BENNINGTON SENATE,DICK SEARS,5086,Registered,False,Leading candidate
24,CALEDONIA SENATE,MATTHEW CHOATE,2881,Registered,False,NaN


In [8]:
prog_house = getrace('PROGRESSIVE', 'STATE REPRESENTATIVE')b


<ipython-input-6-96c5a68314da>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winners['Leading candidate'] = 'Leading candidate'


In [9]:
prog_sen = getrace('PROGRESSIVE', 'STATE SENATOR')

<ipython-input-6-96c5a68314da>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winners['Leading candidate'] = 'Leading candidate'


In [15]:
dem_sen['Party'] = 'DEMOCRATIC'
gop_sen['Party'] = 'REPUBLICAN'
dem_house['Party'] = 'DEMOCRATIC'
gop_house['Party'] = 'REPUBLICAN'
prog_house['Party'] = 'PROGRESSIVE'

In [16]:
all_leg  = pd.concat([dem_sen, gop_sen, dem_house, gop_house, prog_house])

In [244]:
#optional test to try to find contested races
# dist_piv = all_leg.pivot_table(index='OfficeDistrict', columns='Party', values='CandidateId', aggfunc=len)

# single_cand = dist_piv[dist_piv.CandidateId < 2].rename(columns={'CandidateId': 'Single candidate'})


In [245]:
# single_cand = dist_piv[(dist_piv.DEMOCRATIC.isna()) | (dist_piv.REPUBLICAN.isna())]

# single_cand = single_cand[single_cand.PROGRESSIVE.isnull()]



In [246]:
# all_leg.merge(single_cand, left_on='OfficeDistrict', right_index=True).to_csv('test.csv')

### Voter turnout data

In [105]:
# accessing the turnout data is easier, but still not as much fun as just downloading it from the SOS site.
turnout_requests = requests.get('https://electionresults.vermont.gov/rss/4353/TurnoutData.xml', headers=headers)
turnout_xml = turnout_requests.text

with open('TurnoutData.xml', 'w+') as fd:
    fd.write(turnout_xml)

In [106]:
with open('TurnoutData.xml') as fd:
    turnout = xmltodict.parse(fd.read())


In [107]:
turnoutdf = pd.DataFrame(turnout['rss']['channel']['TurnoutData']['Town']).set_index('Id')

In [108]:
turnoutdf.head()

,Town,County,District,Registeredvoters,AbsenteeBallots,AccessibleVotingSystem,DefectiveBallots,CountedBallots,ProvisionalBallots,TotalBallotscast,UpdateRecordStamp,IsLock
Id,,,,,,,,,,,,
16,ADDISON,ADDISON,ADDISON 3,1104,270,0,15,398,0,0,2020-08-12 10:17:31,false
17,ALBANY,ORLEANS,ORLEANS-CALEDONIA,652,187,0,10,272,0,0,2020-08-11 21:54:31,false
18,ALBURGH,GRAND ISLE,GRAND ISLE-CHITTENDEN,1461,254,0,19,413,0,0,2020-08-12 09:34:16,true
19,ANDOVER,WINDSOR,WINDSOR 3-1,429,93,0,6,185,0,0,2020-08-12 09:48:15,true
20,ARLINGTON,BENNINGTON,BENNINGTON 4,2008,387,0,28,626,0,0,2020-08-11 19:50:46,false


In [109]:
# turnoutdf.to_csv('turnout_data.csv')

### Full results 

In [199]:
# as you can see I never really bothered with this one.


# https://electionresults.vermont.gov/rss/4353/ResultsData.xml

full_requests = requests.get('https://electionresults.vermont.gov/rss/4353/ResultsData.xml', headers=headers)
full_xml = full_requests.text

with open('FullResults.xml', 'w+') as fd:
    fd.write(full_xml)

In [200]:
with open('FullResults.xml') as fd:
    full = xmltodict.parse(fd.read())


In [202]:
# full['rss']['channel']['ResultsData']['Town'][0]